In [1]:
from langgraph.graph import StateGraph,START,END
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from typing import TypedDict

In [2]:
load_dotenv()

True

In [18]:
model = ChatGoogleGenerativeAI(
    model='gemini-2.5-flash',
    temperature=0
)

In [19]:
class BlogState(TypedDict):

    title:str
    outline:str
    content:str
    evalution:str

In [20]:
def create_outline(state:BlogState)->BlogState:
    title=state['title']

    prompt=f'Generate a detail outline for a blog on the topic -{title}'

    outline=model.invoke(prompt).content

    state['outline']=outline

    return state

In [21]:
def create_blog(state:BlogState)->BlogState:

    title=state['title']
    outline=state['outline']

    prompt=f'Write a detail a blog on the title-{title} using following outline \n {outline}'

    content=model.invoke(prompt).content

    state['content']=content

    return state

In [22]:
def evaluate_blog(state:BlogState)->BlogState:

    outline=state['outline']
    content=state['content']

    prompt=f'evaluate my blog -{content} based on my give outline -{outline} and give a details analysis and final score of this blog out of 100%'

    evalution=model.invoke(prompt)

    state['evalution']=evalution

    return state

In [24]:
graph=StateGraph(BlogState)

# nodes
graph.add_node('create_outline',create_outline)
graph.add_node('create_blog',create_blog)
graph.add_node('evaluate_blog',evaluate_blog)

#edges
graph.add_edge(START,'create_outline')
graph.add_edge('create_outline','create_blog')
graph.add_edge('create_blog','evaluate_blog')
graph.add_edge('evaluate_blog',END)

# complie
workflow=graph.compile()

In [ ]:
intial_state={'title':'Cricket'}
final_state=workflow.invoke(intial_state)

print(final_state['evalution'])